In [ ]:
import os
# import numpy as np
# from multiprocessing.dummy import Pool as ThreadPool
# from multiprocessing import cpu_count
# from datetime import datetime
import glob
import h5py
# import pandas as pd

import pickle
import numpy as np
import pandas as pd
import gzip
import pickle
import _pickle as cPickle
import glob

In [ ]:


def concatenate(file_names, year):
    temp_h5_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/virt_ds/{year}.h5'
    entry_key = 'precipitationCal'  # where the data is inside of the source files.
    
    fn_1 = file_names[0]
    # dt = fn_1[128:143]
    out_fn = temp_h5_fmt.format(year=year)
    if os.path.exists(out_fn):
        return out_fn
    # fn_out = temp_fn.format(month=str(month).zfill(2))
    # if os.path.exists(out_path)==False:
    #     os.makedirs(out_path)
    # full_fn_out = os.path.join(out_path,fn_out)


    sh = h5py.File(fn_1, 'r')[entry_key].shape  # get the first ones shape.
    layout = h5py.VirtualLayout(shape=(len(file_names),) + sh,
                                dtype=float)
    with h5py.File(out_fn, 'w', libver='latest') as f:
        for i, filename in enumerate(file_names):
            vsource = h5py.VirtualSource(filename, entry_key, shape=sh)
            layout[i, :, :, :] = vsource

        f.create_virtual_dataset(entry_key, layout, fillvalue=-9999.9)
    return out_fn

In [ ]:
fn_list = sorted(glob.glob('/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/time/2000/*.h5'))

fn_in = concatenate(fn_list, 2000)
# fn_in = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/time/2000/07.h5'
year = 2000

h5fr = h5py.File(fn_in,'r') 
# /storage/coda1/p-rbras6/0/njadidoleslam3/gpm/time/2000
# %timeit h5fr['precipitationCal'][:,0]
# %timeit 
# print(h5fr['precip'].chunks)
# %timeit h5fr['precipitationCal'][0,0,:]
dset = h5fr['precipitationCal']
# %timeit a = h5fr['precipitationCal'][:,:,2,2]

In [ ]:
%timeit a = np.ravel(dset[:,:,0,500])



In [ ]:
# import matplotlib.pyplot as plt
%timeit a = dset[:,:,0,500]
# plt.plot(a[0])
# b = []
%timeit b = np.ravel(a)



In [ ]:
len(dset[:,:,0,500][0])

In [ ]:


import numpy as np
import pickle
import itertools
import h5py
import glob
import timeit
import os
#######################

def extract_events(year):
    def storm_def_new(p_data, mit):
        v = (p_data==0)*1
        n = v==0
        a = ~n
        c = np.cumsum(a)
        d = np.diff(np.append([0.], c[n]))
        v[n] = -d
        dry_vec = np.cumsum(v)
        # mit = 6
        aa = np.append([1], np.diff(dry_vec))
        idx = (aa  <=-mit)*(aa<0)
        dry_periods = aa[idx]*-1
        return np.array([mit, np.std(dry_periods[1:]) / np.mean(dry_periods[1:]), np.mean(dry_periods[1:])])

    #######################

    def concatenate(year):
        fn_list_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/time/{year}/*.h5'
        file_names = sorted(glob.glob(fn_list_fmt.format(year = year)))
        temp_h5_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/virt_ds/{year}.h5'
        entry_key = 'precipitationCal'  # where the data is inside of the source files.
    
        fn_1 = file_names[0]
        out_fn = temp_h5_fmt.format(year=year)
        if os.path.exists(out_fn):
            print('file exists')
            return out_fn

        sh = h5py.File(fn_1, 'r')[entry_key].shape  # get the first ones shape.
        layout = h5py.VirtualLayout(shape=(len(file_names),) + sh, dtype=float)
        with h5py.File(out_fn, 'w', libver='latest') as f:
            for i, filename in enumerate(file_names):
                vsource = h5py.VirtualSource(filename, entry_key, shape=sh)
                layout[i, :, :, :] = vsource

            f.create_virtual_dataset(entry_key, layout, fillvalue=-9999.9)
        f.close()
        return out_fn

    #######################

    # @jit('(f4[:],int32)')
    def calc_min_mit(p_events):
        idx, = np.where((np.diff(np.sign(p_events[:,1]-1)) != 0)*1==1)
        if len(idx)==0:
            a = np.abs(p_events[:,1]-1)
            idx, = np.where(a == a.min())
            return p_events[:,0][idx][0]
        return (p_events[:,0][idx][0]+p_events[:,0][idx+1][0])/2.0

    #######################

    def get_events(p_data,set_min_mit):
        mit_list = [x for x in range(set_min_mit,12*24, 6)]
        mit_dry = []
        for mit in mit_list:
            mit_dry.append(storm_def_new(p_data, mit))
            
        mit_dry = np.array(mit_dry)
        min_mit = calc_min_mit(mit_dry)
        event_metrics = storm_def_new(p_data, min_mit)
        # print(event_metrics)
        return event_metrics
    
    set_min_mit = 1
    fn_in = concatenate(year)
    # %timeit f = h5py.File(fn_in,'r') 
    f = h5py.File(fn_in,'r') 
    # %timeit dset = f['precipitationCal']
    dset = f['precipitationCal']
    # print(dset)
    out_pickle_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/precipitation/gpm/events/{year}.pickle'
    fn_out_pickle = out_pickle_fmt.format(year=year)

    ####
    ## NOTE Grid_x and Grid_y in GPM
    ## for STAGE 4 it is grid_y and grid_x
    # list of grid_xy tuples: [(grid_x, grid_y), ...,(grid_x, grid_y)] where grid_x varies from 0 to 1120 and grid_y varies from 0 to 880
    ##########
    # perm_list = np.array([itertools.product(np.arange(3600),np.arange(1800))])[500000:500100]##
    ####

    perm_list = [(500,1000), (500, 1001)] ## Comment this line for full run
    
    result = np.empty((1,1), dtype=object)
    for grid_xy in perm_list:
        grid_x, grid_y = grid_xy
        gid = int('1{gid_x}{gid_y}'.format(gid_x = str(grid_x).zfill(4), gid_y = str(grid_y).zfill(4)))
        
        p_data = np.ravel(dset[:, :, grid_x, grid_y])
        print(p_data)
        # %timeit p_data1 =  p_data.reshape((int(p_data.shape[0]/2),2)).mean(axis=1)
        p_data =  p_data.reshape((int(p_data.shape[0]/2),2)).mean(axis=1)
        n_data = len(p_data)
        # event_metrics = get_events(p_data, set_min_mit)
        # %timeit np.sum(p_data < 0)
        pos_idx = np.sum(p_data>0)
        neg_idx = ~pos_idx
        if float(np.sum(neg_idx)/n_data)<0.1:
            # print('No data 1', float(np.sum(p_data < 0)/n_data))
            p_data[p_data<0] = 0
            
        if (float(np.sum(pos_idx)/n_data)<0.10):
            result = np.append(result,np.array([gid,np.nan,np.nan,np.nan]))
            # print('No data 2')
        else:
            try:
                event_metrics = get_events(p_data, set_min_mit)
                result = np.append(result,np.concatenate((gid, event_metrics),axis=None))
            except:
                result = np.append(result,np.array([gid,np.nan,np.nan,np.nan]))

    
    with open(fn_out_pickle, 'wb') as handle:
        pickle.dump(result, handle, protocol= pickle.HIGHEST_PROTOCOL)
    # return result

year = 2002
extract_events(year)


In [ ]:
set_min_mit = 1
import numpy as np
import pickle
import itertools
import h5py
import glob
import timeit
def concatenate(year):
    fn_list_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/time/{year}/*.h5'
    file_names = sorted(glob.glob(fn_list_fmt.format(year = year)))
    temp_h5_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/virt_ds/{year}.h5'
    entry_key = 'precipitationCal'  # where the data is inside of the source files.

    fn_1 = file_names[0]
    out_fn = temp_h5_fmt.format(year=year)
    if os.path.exists(out_fn):
        print('file exists')
        return out_fn

    sh = h5py.File(fn_1, 'r')[entry_key].shape  # get the first ones shape.
    layout = h5py.VirtualLayout(shape=(len(file_names),) + sh, dtype=float)
    with h5py.File(out_fn, 'w', libver='latest') as f:
        for i, filename in enumerate(file_names):
            vsource = h5py.VirtualSource(filename, entry_key, shape=sh)
            layout[i, :, :, :] = vsource

        f.create_virtual_dataset(entry_key, layout, fillvalue=-9999.9)
    f.close()
    return out_fn
year = 2003
fn_in = concatenate(year)
# %timeit f = h5py.File(fn_in,'r') 
# fn_in = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/time/2001/01.h5'
f = h5py.File(fn_in,'r') 
# %timeit dset = f['precipitationCal']
dset = f['precipitationCal']
print(dset)
out_pickle_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/precipitation/gpm/events/{year}.pickle'
fn_out_pickle = out_pickle_fmt.format(year=year)

In [ ]:
sum(dset[:,:, 120, 1012])

In [1]:
import xarray
# dset = xarray.open_mfdataset('/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/time/2001/*.h5',  combine = 'nested', concat_dim='phony_dim_0', parallel=True)
dset = xarray.open_dataset('/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/time/2001/01.h5')

In [43]:
import numpy as np
import h5py
from scipy import spatial
from netCDF4 import Dataset
# def get_grid_x_y(target_pts):
#     fn = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2000/366/3B-HHR.MS.MRG.3IMERG.20001231-S233000-E235959.1410.V06B.HDF5'
#     dset = h5py.File(fn,'r')
#     lat  = dset['Grid/lat'][:]
#     lon  = dset['Grid/lon'][:]
#     # make them a meshgrid for later use KDTree
#     lon2d, lat2d = np.meshgrid(lon, lat)
#     # zip them together
#     model_grid = list( zip(np.ravel(lon2d), np.ravel(lat2d)) )
#     #target point location : 30.5N, 56.1E
#     distance, index = spatial.KDTree(model_grid).query(target_pts)
#     # the nearest model location (in lat and lon)
#     model_loc_coord = [coord for i, coord in enumerate(model_grid) if i==index]

#     print(model_loc_coord,index)
#     # return lat_grid[i,j], lon_grid[i,j]

def get_grid_x_y(in_lat, in_lon):
    def geo_idx(dd, dd_array):
        """
        search for nearest decimal degree in an array of decimal degrees and return the index.
        np.argmin returns the indices of minium value along an axis.
        so subtract dd from all values in dd_array, take absolute value and find index of minium.
        """
        geo_idx = (np.abs(dd_array - dd)).argmin()
        return geo_idx

    fn = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2000/366/3B-HHR.MS.MRG.3IMERG.20001231-S233000-E235959.1410.V06B.HDF5'
    dset = h5py.File(fn,'r')
    lats  = dset['Grid/lat'][:]
    lons  = dset['Grid/lon'][:]

    lat_idx = geo_idx(in_lat, lats)
    lon_idx = geo_idx(in_lon, lons)
    return lat_idx, lon_idx

in_lat, in_lon = [30.5,56.1]
lat_idx, lon_idx = get_grid_x_y(in_lat, in_lon)

(1204, 2360)

In [106]:
import pandas as pd
calval_sites = pd.read_csv('/storage/home/hcoda1/6/njadidoleslam3/p-rbras6-0/projects/stochsm/data/calval_sites.csv', delimiter=',')

In [107]:
type(calval_sites)

pandas.core.frame.DataFrame

In [99]:
calval_sites

,oid,Site Name,Site ID,Site PI,Location,"Latitude, Longitude (approximate)",Core Site Scales [km]I,Climate RegimeII,IGBPIII Land Cover,References,lat,lon
0,1,Tonzi Ranch,2501,M. Moghaddam,USA (California),"38.43° N, 120.97° W",(Not a core site),Temperate,Savannas woody,"Clewley et al., 2017",38.430,-120.970
1,2,Walnut Gulch*,1601,D. C. Goodrich,USA (Arizona),"31.72° N, 110.68° W","3, 9, 36",Arid,Shrub open,"Keefer et al., 2008",31.720,-110.680
2,3,Reynolds Creek*,401,M. Seyfried,USA (Idaho),"31.72° N, 110.68° W","9, 36",Arid,Grasslands,"Seyfried et al., 2001",31.720,-110.680
3,4,TxSON**,4801,T. Caldwell,USA (Texas),"30.5° N, 98.5° W","3, 9, 36",Temperate,Grasslands,(Not available),30.500,-98.500
4,5,Fort Cobb*,1603,P. J. Starks,USA (Oklahoma),"35.36° N, 98.55° W",36,Temperate,Grasslands,(Not available),35.360,-98.550
5,6,Little Washita*,1602,P. J. Starks,USA (Oklahoma),"34.97° N, 97.97° W","9, 36",Temperate,Grasslands,"Cosh et al., 2006",34.970,-97.970
6,7,South Fork*,1607,M. H. Cosh/J. Prueger,USA (Iowa),"42.44° N, 93.44° W","3, 9, 36",Cold,Croplands,"Coopersmith et al., 2015",42.440,-93.440
7,8,St. Josephs*,1606,S. Livingston,USA (Indiana),"41.449° N, 85.011° W",9,Cold,Croplands,(Not available),41.449,-85.011
8,9,Little River*,1604,D. Bosch,USA (Georgia),"31.64° N, 83.65° W","3, 9, 36",Temperate,Cropland/natural mosaic,"Bosch et al., 2007",31.640,-83.650
9,10,Millbrook,2601,M. Temimi,USA (New York),"41.78° N, 73.73° W",(Not a core site),Cold,Forest deciduous broadleaf,(Not available),41.780,-73.730


In [109]:
for i in range(len(calval_sites)):
    print(calval_sites.loc[i]['oid'])


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [54]:
calval_sites

,oid,Site Name,Site ID,Site PI,Location,"Latitude, Longitude (approximate)",Core Site Scales [km]I,Climate RegimeII,IGBPIII Land Cover,References,lat,lon
0,1,Tonzi Ranch,2501,M. Moghaddam,USA (California),"38.43° N, 120.97° W",(Not a core site),Temperate,Savannas woody,"Clewley et al., 2017",38.430,-120.970
1,2,Walnut Gulch*,1601,D. C. Goodrich,USA (Arizona),"31.72° N, 110.68° W","3, 9, 36",Arid,Shrub open,"Keefer et al., 2008",31.720,-110.680
2,3,Reynolds Creek*,401,M. Seyfried,USA (Idaho),"31.72° N, 110.68° W","9, 36",Arid,Grasslands,"Seyfried et al., 2001",31.720,-110.680
3,4,TxSON**,4801,T. Caldwell,USA (Texas),"30.5° N, 98.5° W","3, 9, 36",Temperate,Grasslands,(Not available),30.500,-98.500
4,5,Fort Cobb*,1603,P. J. Starks,USA (Oklahoma),"35.36° N, 98.55° W",36,Temperate,Grasslands,(Not available),35.360,-98.550
5,6,Little Washita*,1602,P. J. Starks,USA (Oklahoma),"34.97° N, 97.97° W","9, 36",Temperate,Grasslands,"Cosh et al., 2006",34.970,-97.970
6,7,South Fork*,1607,M. H. Cosh/J. Prueger,USA (Iowa),"42.44° N, 93.44° W","3, 9, 36",Cold,Croplands,"Coopersmith et al., 2015",42.440,-93.440
7,8,St. Josephs*,1606,S. Livingston,USA (Indiana),"41.449° N, 85.011° W",9,Cold,Croplands,(Not available),41.449,-85.011
8,9,Little River*,1604,D. Bosch,USA (Georgia),"31.64° N, 83.65° W","3, 9, 36",Temperate,Cropland/natural mosaic,"Bosch et al., 2007",31.640,-83.650
9,10,Millbrook,2601,M. Temimi,USA (New York),"41.78° N, 73.73° W",(Not a core site),Cold,Forest deciduous broadleaf,(Not available),41.780,-73.730


In [91]:
# lat_lons = [(row['lat'], row['lon']) for i,row in enumerate(calval_sites)]
import pandas as pd
import h5py
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import cpu_count
calval_sites = pd.read_csv('/storage/home/hcoda1/6/njadidoleslam3/p-rbras6-0/projects/stochsm/data/calval_sites.csv', delimiter=',')

def get_grid_x_y(in_lat, in_lon):
    def geo_idx(dd, dd_array):
        """
        search for nearest decimal degree in an array of decimal degrees and return the index.
        np.argmin returns the indices of minium value along an axis.
        so subtract dd from all values in dd_array, take absolute value and find index of minium.
        """
        geo_idx = (np.abs(dd_array - dd)).argmin()
        return geo_idx

    fn = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2000/366/3B-HHR.MS.MRG.3IMERG.20001231-S233000-E235959.1410.V06B.HDF5'
    dset = h5py.File(fn,'r')
    lats  = dset['Grid/lat'][:]
    lons  = dset['Grid/lon'][:]

    lat_idx = geo_idx(in_lat, lats)
    lon_idx = geo_idx(in_lon, lons)
    return lat_idx, lon_idx

calval_sites = pd.read_csv('/storage/home/hcoda1/6/njadidoleslam3/p-rbras6-0/projects/stochsm/data/calval_sites.csv', delimiter=',')
fn_temp  = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/time/{year}/{month}.h5'
path_out_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/calval_sites/{year}/{month}/'
fn_out_fmt = '{site_id}.csv'
def get_data(year):
    for month in range(1,13):
        out_path = path_out_fmt.format(year = year, month=str(month).zfill(2))
        if not os.path.exists(out_path):
            os.makedirs(out_path)


        fn = fn_temp.format(year = year, month=str(month).zfill(2))
        f = h5py.File(fn,'r')
        dset = f['precipitationCal']
        
        time = f['time'][:]
        t = time.reshape((int(time.shape[0]/2),2))[:,0]
        dt = pd.to_datetime(t, unit='s')
        for i in range(len(calval_sites)):
            in_lat = calval_sites.loc[i]['lat']
            in_lon = calval_sites.loc[i]['lon']
            lat_idx, lon_idx = get_grid_x_y(in_lat, in_lon)
            p_data = dset[:, lon_idx,lat_idx ]
            p_data =  p_data.reshape((int(p_data.shape[0]/2),2)).mean(axis=1)
            fn_out = os.path.join(out_path, fn_out_fmt.format(site_id = calval_sites.loc[i]['Site ID']))
            df = pd.DataFrame({'dt':dt, 'p':p_data }).to_csv(fn_out)
if __name__=='__main__':
    n_cpu = cpu_count()
    print('# of CPUS working: ' + str(n_cpu))
    pool = ThreadPool(n_cpu)
    permute_list = list([x for x in range(2001,2021)])
    pool.map(get_data, permute_list)

KeyboardInterrupt: 

In [89]:
pd.to_datetime(t, unit='s')

DatetimeIndex(['2020-02-01 00:00:00', '2020-02-01 01:00:00',
               '2020-02-01 02:00:00', '2020-02-01 03:00:00',
               '2020-02-01 04:00:00', '2020-02-01 05:00:00',
               '2020-02-01 06:00:00', '2020-02-01 07:00:00',
               '2020-02-01 08:00:00', '2020-02-01 09:00:00',
               ...
               '2020-02-29 14:00:00', '2020-02-29 15:00:00',
               '2020-02-29 16:00:00', '2020-02-29 17:00:00',
               '2020-02-29 18:00:00', '2020-02-29 19:00:00',
               '2020-02-29 20:00:00', '2020-02-29 21:00:00',
               '2020-02-29 22:00:00', '2020-02-29 23:00:00'],
              dtype='datetime64[ns]', length=696, freq=None)

In [63]:
t = time.reshape((int(time.shape[0]/2),2))[:,0]

In [78]:
np.diff(t)

array([3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 36

In [ ]:
perm_list = [(500,1000), (500, 1001)] ## Comment this line for full run
    
result = np.empty((1,1), dtype=object)
for grid_xy in perm_list:
    p_data = np.ravel(dset[:, :, 500, 1000])

In [ ]:
result[3]

In [ ]:
np.shape(p_data)

In [ ]:
np.negative(p_data).shape[0]

In [ ]:
def concatenate(year):
    fn_list_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/time/{year}/*.h5'
    file_names = sorted(glob.glob(fn_list_fmt.format(year = year)))
    temp_h5_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/gpm/virt_ds/{year}.h5'
    entry_key = 'precipitationCal'  # where the data is inside of the source files.

    fn_1 = file_names[0]
    out_fn = temp_h5_fmt.format(year=year)
    if os.path.exists(out_fn):
        return out_fn

    sh = h5py.File(fn_1, 'r')[entry_key].shape  # get the first ones shape.
    layout = h5py.VirtualLayout(shape=(len(file_names),) + sh, dtype=float)
    with h5py.File(out_fn, 'w', libver='latest') as f:
        for i, filename in enumerate(file_names):
            vsource = h5py.VirtualSource(filename, entry_key, shape=sh)
            layout[i, :, :, :] = vsource

        f.create_virtual_dataset(entry_key, layout, fillvalue=-9999.9)
    return out_fn

fn_in = concatenate(year=2001)
f = h5py.File(fn_in,'r') 
dset = f['precipitationCal']
p_data = np.ravel(dset[:, :, 500,1000])

out_pickle_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/precipitation/gpm/events/{year}.pickle'
fn_out_pickle = out_pickle_fmt.format(year=year)
p_data1 =  x.reshape((int(x.shape[0]/2),2)).mean(axis=1)

In [ ]:
import numpy as np
x = np.array([0.01, 0.09,1, 0.04, 0.16, 2])
# reshaping the array to a 2D array, and summing over the columns
x.reshape((int(x.shape[0]/2),2)).mean(axis=1)

In [ ]:
x.reshape((int(x.shape[0]/2),2))

In [ ]:
result = np.empty((1,1))
np.append(result, [1,2,3,4])